###### Importing libraries

In [8]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
% matplotlib inline

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


###### Reading in the data to model

In [14]:
X_test = pd.read_csv('../Data/test_x.csv')
X = pd.read_csv('../Data/train_x.csv')
y = pd.read_csv('../Data/train_y.csv', header = None)
print(X_test.shape)
print(X.shape)
print(y.shape)

(1459, 111)
(1445, 111)
(1445, 1)


###### Gradient Boosting (book, p. 198)

In [21]:
type(np.ravel(y))

numpy.ndarray

In [114]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, np.ravel(y))

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=42)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators, random_state=42)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=298, presort='auto', random_state=42,
             subsample=1.0, verbose=0, warm_start=False)

In [115]:
bst_n_estimators  # 270 (OUT OF 300)

298

In [116]:
min(errors) # 0.014635885043588544

0.012948454985639325

In [117]:
min(errors)**0.5

0.11379127816155035

In [101]:
errors[125]**0.5  # 0.12097886

0.12598064703516204

###### Early Stopping

In [85]:
# Doesn't work well - error is too large
gbrt = GradientBoostingRegressor(max_depth=4, warm_start=True, random_state=42)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 300):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # early stopping

In [86]:
min_val_error

0.032939211594853912

In [87]:
n_estimators

15